In [ ]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from quick_pp.objects import Project

# Load well from saved file
project = "MOCK_carbonate"
project_path = rf"data\04_project\{project}.qppp"
project = Project().load(project_path)
project.get_well_names()

In [ ]:
# Load data
well_name = 'HW-24'
well = project.get_well(well_name)
well_data = well.data.copy()
well_data['CPORE'] = well_data['CORE_POR'] / 100
well_data['CPERM'] = well_data['CORE_PERM']
# well_data.dropna(subset=['NPHI', 'RHOB', 'RT', 'GR'], inplace=True)

# Quick PP - Single Well Interpretation

In [68]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from quick_pp.lithology.carbonate import Carbonate
from quick_pp.porosity import neu_den_xplot_poro, density_porosity, rho_matrix
from quick_pp.qaqc import badhole_flagging, mask_outside_threshold, neu_den_xplot_hc_correction
from quick_pp.rock_type import rock_typing, estimate_vsh_gr
from quick_pp.plotter.plotter import plotly_log, neutron_density_xplot

## Estimate Litholoy

In [69]:
# Define parameters
args = {
    'litho_model': 'carb',
    'dry_calc_point': (.0, 2.71),
    'fluid_point': (1.0, 1.0),
    # 'dry_dolo_point': (.0, 2.81),
    'dry_clay_point': (.3, 2.7),
    'silt_line_angle': 116,
    'wet_clay_point': (0.43, 2.6),
    'sw_water_salinity': 2e5,
    'sw_m': 2.2,
    'sw_n': 2,
    'hc_corr_angle': 50,
    'hc_buffer': 0.1,
    'ressum_cutoffs': dict(
        VSHALE=.5,
        PHIT=0,
        SWT=1
    ),
}

In [ ]:
carbonate_type =  'limestone'  # 'dolostone'  #
model = 'single'  # 'double'  #
method = 'neu_den'  # 'pef_den'  #

# Clean up data
well_data = badhole_flagging(well_data)
# well_data = mask_outside_threshold(well_data, fill=True)

vsh_gr = estimate_vsh_gr(well_data['GR'], min_gr=0, max_gr=150)
carb_model = Carbonate(**args)
vclw, vcalc, vdolo = carb_model.estimate_lithology(
    well_data['NPHI'], well_data['RHOB'], vsh_gr=vsh_gr,  # pef=well_data['PEF'],
    model=model, method=method, carbonate_type=carbonate_type
)
args.update(carb_model.__dict__)
well.update_config(args)  # Save lithology model to well

# Set skip HC correction or not
skip_hc_correction = False
if skip_hc_correction:
    nphihc, rhobhc = well_data['NPHI'], well_data['RHOB']
else:
    # Implement hydrocarbon correction
    nphihc, rhobhc, hc_flag = neu_den_xplot_hc_correction(
        well_data['NPHI'], well_data['RHOB'], vsh_gr=vsh_gr,
        dry_min1_point=args['dry_calc_point'],
        dry_clay_point=args['dry_clay_point'],
        corr_angle=args['hc_corr_angle'], buffer=args['hc_buffer']
    )
    # Estimate lithology
    carb_model = Carbonate(**args)
    vclw, vcalc, vdolo = carb_model.estimate_lithology(
        nphihc, rhobhc,  vsh_gr=vsh_gr,  # pef=well_data['PEF'], 
        model=model, method=method, carbonate_type=carbonate_type
    )

In [ ]:
neutron_density_xplot(well_data['NPHI'], well_data['RHOB'], dry_min1_point=args['dry_calc_point'], **args)

In [ ]:
neutron_density_xplot(nphihc, rhobhc, dry_min1_point=args['dry_calc_point'], **args)

## Estimate Porosity

In [ ]:
# Estimate porosity
phit = neu_den_xplot_poro(
    nphihc, rhobhc, model='carb',
    dry_min1_point=args['dry_calc_point'],
    dry_clay_point=args['dry_clay_point'],
)

# PHID needs unnormalized lithology
vclw_un, vcalc_un, vdolo_un = Carbonate(**args).estimate_lithology(
    nphihc, rhobhc, vsh_gr=vsh_gr,  # pef=well_data['PEF'],
    model=model, method=method, carbonate_type=carbonate_type
)
rho_ma = rho_matrix(vclay=vclw_un, vcalc=vcalc_un, vdolo=vdolo_un)
phid = density_porosity(rhobhc, rho_ma)

plt.figure(figsize=(20, 2))
plt.plot(phit, label='PHIT')
plt.plot(phid, label='PHID')
plt.legend()

## Predict Permeability and Rock Typing

In [ ]:
import pickle
from sklearn.preprocessing import MinMaxScaler
from quick_pp.rock_type import calc_r35_perm, calc_fzi_perm
from quick_pp.core_calibration import perm_transform

poroperm_params = {
    1: (6800, 1.9),
    2: (1400, 2.0),
    3: (110, 2.0),
    4: (17, 2.0)
}

well_data['LOG_RT'] = np.log10(well_data['RT'])
well_data['NDI'] = (2.95 - well_data['RHOB']) / 1.95
well_data['GRN'] = MinMaxScaler().fit_transform(well_data[['GR']])
input_features = ['GR', 'NPHI', 'RHOB', 'NDI', 'RT', 'LOG_RT', 'GRN']

# Predict ROCK_FLAG
with open(r'data\04_project\MOCK_carbonate\outputs\fzi_rt_model.qppm', 'rb') as file:
    rt_model = pickle.load(file)
rock_flag_ml = rt_model.predict(well_data[input_features])

# Predict PERM based on predicted rock type
perm_a_ml = pd.Series(rock_flag_ml).map(poroperm_params).apply(lambda x: x[0] if type(x) == tuple else np.nan)
perm_b_ml = pd.Series(rock_flag_ml).map(poroperm_params).apply(lambda x: x[1] if type(x) == tuple else np.nan)
perm_rt_ml = perm_transform(phit, perm_a_ml, perm_b_ml)

use_r35 = False
if use_r35:
    # Predict permeability
    # Load R35 model
    with open(r'data\04_project\MOCK_carbonate\outputs\r35_model.qppm', 'rb') as file:
        r35_model = pickle.load(file)
    r35_ml = 10**(r35_model.predict(well_data[input_features]))
    perm_ml = calc_r35_perm(r35_ml, phit)

    # Determine rock type based on perm_r35_ml
    r35_cut_offs = [
        0.15, .2, .3, .4, .5, .6, .7, .8, .9, 1, 1.25, 1.5, 2, 2.5, 3, 4, 5, 7, 10,
    ]
    rock_flag_r35 = rock_typing(r35_ml, higher_is_better=True, cut_offs=r35_cut_offs)
    well_data['ROCK_FLAG'] = rock_flag_r35
    shf_params = {   
        1: (0.006, 1.212),
        2: (0.004, 1.733),
        3: (0.008, 1.465),
        4: (0.012, 1.513),
        5: (0.005, 1.6),
        6: (0.015, 0.688),
        7: (0.016, 0.932),
        8: (0.024, 0.585),
        9: (0.002, 2.136),
        10: (0.015, 1.052),
        11: (0.013, 0.819),
        12: (0.006, 2.057),
        13: (0.012, 0.861),
        14: (0.012, 0.797),
        15: (0.018, 1.457),
        16: (0.007, 1.21),
        17: (0.026, 0.773),
        18: (0.018, 0.927),
        19: (0.021, 0.649),
        20: (0.026, 1.033),
        21: (0.023, 0.922),
        22: (0.008, 1.279),
        23: (0.025, 0.581)
    }
else:
    # Predict permeability
    # Load FZI model
    with open(r'data\04_project\MOCK_carbonate\outputs\fzi_model.qppm', 'rb') as file:
        fzi_model = pickle.load(file)
    well_data['ROCK_PRED'] = rock_flag_ml
    input_features = ['GR', 'NPHI', 'RHOB', 'NDI', 'RT', 'LOG_RT', 'GRN', 'ROCK_PRED']
    fzi_ml = 10**(fzi_model.predict(well_data[input_features]))
    perm_ml = calc_fzi_perm(fzi_ml, phit)

    # Determine rock type based on perm_fzi_ml
    fzi_cut_offs = [
        .15, .2, .25, .3, .35, .4, .45, .5, .6, .7, .8, .9, 1.1, 1.3, 1.5, 2, 2.5, 3, 4, 5, 7, 10
    ]
    rock_flag_fzi = rock_typing(fzi_ml, higher_is_better=False, cut_offs=fzi_cut_offs)
    well_data['ROCK_FLAG'] = rock_flag_fzi
    shf_params = {   
        1: (0.009, 1.033),
        2: (0.026, 0.978),
        3: (0.006, 1.578),
        4: (0.022, 0.566),
        5: (0.032, 0.387),
        6: (0.022, 0.513),
        7: (0.015, 0.659),
        8: (0.018, 0.728),
        9: (0.02, 0.85),
        10: (0.033, 0.572),
        11: (0.03, 0.52),
        12: (0.024, 0.761),
        13: (0.025, 0.658),
        14: (0.028, 0.513),
        15: (0.028, 0.579),
        16: (0.008, 1.615),
        17: (0.012, 1.136),
        18: (0.009, 1.494),
        19: (0.021, 0.944),
        20: (0.036, 0.493),
        21: (0.013, 0.853),
        22: (0.013, 1.121),
        23: (0.012, 1.341)
    }



plt.figure(figsize=(20, 2))
plt.plot(well_data.DEPTH, perm_rt_ml, label='Perm Rock Flag ML')
plt.plot(well_data.DEPTH, perm_ml, label='Perm R35 ML')
plt.scatter(well_data.DEPTH, well_data.CPERM, label='Core Perm', marker='.', c='black')
plt.yscale('log')
plt.legend()

In [ ]:

rock_test = rock_typing(fzi_ml, higher_is_better=True, cut_offs=fzi_cut_offs)
plt.figure(figsize=(20, 1))
plt.plot(well_data.DEPTH, rock_flag_ml, label='Rock Flag ML')
plt.plot(well_data.DEPTH, rock_test, label='Rock Flag Test')
plt.legend()
plt.figure(figsize=(20, 1))
plt.plot(well_data.DEPTH, fzi_ml, label='FZI ML')
plt.ylim(0, 10)
plt.legend()
plt.figure(figsize=(20, 1))
plt.plot(well_data.DEPTH, well_data.GR, label='GR')

## Estimate Water Saturation

In [76]:
from quick_pp.saturation import estimate_rw_temperature_salinity, archie_saturation, estimate_temperature_gradient

# Estimate log derived water saturation
water_salinity = args['sw_water_salinity']
m = args['sw_m']
temp_grad = estimate_temperature_gradient(well_data['DEPTH'], 'imperial')
rw = estimate_rw_temperature_salinity(temp_grad, water_salinity)
swt = archie_saturation(well_data['RT'], rw, phit, m=m)
swt = swt.clip(0, 1)

In [ ]:
from ipywidgets import interact, widgets

from quick_pp.core_calibration import sw_shf_leverett_j

# Estimate SHF
shf_params = {   
    1: (0.009, 1.033),
    2: (0.026, 0.978),
    3: (0.006, 1.578),
    4: (0.022, 0.566),
    5: (0.032, 0.387),
    6: (0.022, 0.513),
    7: (0.015, 0.659),
    8: (0.018, 0.728),
    9: (0.02, 0.85),
    10: (0.033, 0.572),
    11: (0.03, 0.52),
    12: (0.024, 0.761),
    13: (0.025, 0.658),
    14: (0.028, 0.513),
    15: (0.028, 0.579),
    16: (0.008, 1.615),
    17: (0.012, 1.136),
    18: (0.009, 1.494),
    19: (0.021, 0.944),
    20: (0.036, 0.493),
    21: (0.013, 0.853),
    22: (0.013, 1.121),
    23: (0.012, 1.341)
}

ift = 32
theta = 30
ghc = .837
gw = 1.135
fwl = 8550

fwl = widgets.FloatSlider(
    value=fwl,
    min=fwl / 1.1,
    max=fwl * 1.1,
    step=1
)

@interact(fwl=fwl)
def plot(fwl):
    a = well_data['ROCK_FLAG'].map(shf_params).apply(lambda x: x[0])
    b = well_data['ROCK_FLAG'].map(shf_params).apply(lambda x: x[1])
    shf = sw_shf_leverett_j(
        well_data['PERM'], well_data['PHIT'], well_data['DEPTH'], gw=gw, ghc=ghc,
        fwl=fwl, ift=ift, theta=theta, a=a, b=b)

    plt.figure(figsize=(20, 1))
    plt.plot(well_data['DEPTH'], swt, label='SWT')
    plt.plot(well_data['DEPTH'], shf, label='SHF')
    plt.ylim(0, 1.2)
    plt.legend()
    plt.figure(figsize=(20, 1))
    plt.plot(well_data['DEPTH'], well_data['ROCK_FLAG'], label='Rock Flag')
    plt.legend()

In [78]:
fwl = 8562

a = pd.Series(rock_flag_ml).map(shf_params).apply(lambda x: x[0])
b = pd.Series(rock_flag_ml).map(shf_params).apply(lambda x: x[1])
shf = sw_shf_leverett_j(
    perm_ml, phit, well_data['DEPTH'], gw=gw, ghc=ghc,
    fwl=fwl, ift=ift, theta=theta, a=a, b=b
)

## Update Data

In [79]:
# Update data in the project
well_data['NPHI_HC'] = nphihc
well_data['RHOB_HC'] = rhobhc
well_data['VCALC'] = vcalc
well_data['VDOLO'] = vdolo
well_data['VCLW'] = vclw
well_data['PHID'] = phid
well_data['RW'] = rw
well_data['M'] = m
well_data['SWT'] = swt
well_data['SHF'] = shf
well_data['PERM'] = perm_ml
well_data['VHC'] = phit * (1 - swt)

In [80]:
from quick_pp.ressum import calc_reservoir_summary

# Estimate reservoir summary
well_data['ZONES'] = 'ALL'
ressum_df = calc_reservoir_summary(
    well_data.DEPTH, vclw, phit, swt, perm_ml,
    zones=well_data['ZONES'], cutoffs=args['ressum_cutoffs'])

In [ ]:
well_data.ROCK_FLAG.unique()

In [ ]:
# Plot the results
fig = plotly_log(well_data, depth_uom=well.depth_uom)
fig.show(config=dict(scrollZoom=True))
# fig.write_html(rf"{well_name}_log.html", config=dict(scrollZoom=True))